In [ ]:
import freesasa
import requests
import os

def download_pdb(pdb_id, save_path):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"PDB file successfully downloaded: {save_path}")
    else:
        print(f"file downloaded failed: {response.status_code}")

def check_freesasa_residues(pdb_file):
    try:
        structure = freesasa.Structure(pdb_file)
        result = freesasa.calc(structure)
        sasa_values = result.residueAreas()
        total_residues = 0
        print('''

Amount of Resolved Residues (powered by freesasa):
        ''')
        
        for chain in sasa_values:
            residues = list(sasa_values[chain].keys())
            total_residues += len(residues)
            print(f"① Detailed Sites in Chain {chain}     : {residues}")
        print(f"\n② Total Amount of Resolved Residues : {total_residues}")
    except Exception as err:
        print(f"Error: {err}")

pdb_id = "1ymg"
pdb_file = f"{pdb_id}.pdb"
download_pdb(pdb_id, pdb_file)
check_freesasa_residues(pdb_file)

if os.path.exists(pdb_file):
    os.remove(pdb_file)


In [ ]:
from Bio import PDB

def extract_single_chain(input_pdb, output_pdb, chain_id="A"):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", input_pdb)   
    io = PDB.PDBIO()
    class ChainSelect(PDB.Select):
        def accept_chain(self, chain):
            return chain.get_id() == chain_id
    
    io.set_structure(structure)
    io.save(output_pdb, ChainSelect())
    print(f"Chain {chain_id} has successfully been extracted to:   {output_pdb}")

input_pdb = r"1ymg.pdb"
output_pdb = r"1ymg_chainA.pdb"
extract_single_chain(input_pdb, output_pdb, chain_id="A")

In [ ]:
import nglview as nv
import freesasa
import os
import pandas as pd
from Bio import PDB
from IPython.display import display

def get_residue_names(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    residue_map = {}

    for model in structure:
        for chain in model:
            for residue in chain:
                if PDB.is_aa(residue, standard=True):
                    chain_id = chain.id.upper()
                    res_id = residue.get_id()
                    res_num = str(res_id[1])
                    if res_id[2] != " ":
                        res_num += res_id[2]
                    res_name = residue.get_resname()
                    residue_map[(chain_id, res_num)] = res_name

    return residue_map

def visualize_pdb(pdb_file):
    if not os.path.exists(pdb_file):
        print(f"error: PDB file '{pdb_file}' does not exist，check file path！")
        return None
    try:
        view = nv.show_file(pdb_file)
        display(view)
        return view
    except Exception as e:
        print(f"An error occurred while visualizing the PDB: {e}")
        return None

def calculate_sasa(pdb_file, output_xlsx="sasa_results.xlsx"):
    if not os.path.exists(pdb_file):
        print(f"erroe: PDB file '{pdb_file}' does not exist，check file path！")
        return

    try:
        structure = freesasa.Structure(pdb_file)
        result = freesasa.calc(structure)
        sasa_values = result.residueAreas()

        residue_names = get_residue_names(pdb_file)

        sasa_list = []

        for chain in sasa_values:
            for resnum, residue_area in sasa_values[chain].items():
                total_sasa = residue_area.total
                res_name = residue_names.get((chain.upper(), str(resnum)), "UNK") 
                sasa_list.append([chain, resnum, res_name, total_sasa])

        df_sasa = pd.DataFrame(sasa_list, columns=["Chain", "R.No.", "Amino Acid", "SASA (Å²)"])
        df_sasa = df_sasa.sort_values(by="SASA (Å²)", ascending=True).reset_index(drop=True)

        df_sasa.to_excel(output_xlsx, index=False)
        print(f"\nSASA calculation completed, see result blank in: {output_xlsx}")

        print("\nSASA calculation result example（Sorted by SASA from smallest to largest）:")
        display(df_sasa)

    except Exception as e:
        print(f"an unexcepted error occurred: {e}")

pdb_file = input("input PDB input-file path (or Enter with using '1ymg_chainA.pdb')：").strip()
if not pdb_file:
    pdb_file = r"1ymg_chainA.pdb"

output_xlsx = input("input Excel output-file path (or Enter with using 'sasa_results.xlsx')：").strip()
if not output_xlsx:
    output_xlsx = "sasa_results.xlsx"

view = visualize_pdb(pdb_file)
calculate_sasa(pdb_file, output_xlsx)